In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# SQL Questions


Nearly every interview for a data analyst or data science position involves *some* sort of question about SQL, since it's the primary access method for many databases.  Please **DO NOT USE PANDAS**, and note you can use SQL here by starting a block with `%%sql`

Example answers are at the end, they're not the only way to do it, so as long as you get the same output you should be fine

In [ ]:
%load_ext sql
%sql sqlite:///testdb.sqlite

In [ ]:
%%sql

DROP TABLE IF EXISTS students;

DROP TABLE IF EXISTS scores;

CREATE TABLE students (
    studentid        INTEGER PRIMARY KEY NOT NULL,
    name              TEXT,
    age               INTEGER NOT NULL,
    state             TEXT
);

CREATE TABLE scores (
    studentid        INTEGER NOT NULL,
    datetaken        DATE NOT NULL,
    math              INTEGER,
    verbal            INTEGER,
    writing           INTEGER
);

INSERT INTO students VALUES (234, 'Mary', 17, 'NY'), (333, 'Boba', 16, 'PA'), 
  (9128, 'Typhoid', 18, 'NY'), (777, 'Fett', 15, 'MD'), (1234, 'Allen', 16, 'CA'), 
  (7899, 'Alain', 17, 'OR'), (9981, 'Alan', 18, 'WA'), (4422, 'Zucher', 19, 'CA'),
  (8675309, 'Jenny', 17, 'CA'), (666, 'Lou', 20, 'PA'), (5555, 'Daft', 17, 'WA');
    
INSERT INTO scores VALUES (234, '2016-06-01', 650, 720, NULL), 
   (234, '2016-05-17', 600, 550, 750), (234, '2016-04-12', 710, 500, 300), 
   (333, '2016-07-04', 650, 680, NULL), (333, '2016-06-04', 500, NULL, 670),
   (9128, '2016-08-07', NULL, NULL, 650), (9128, '2016-05-05', 720, NULL, NULL),
   (777, '2016-12-25', 650, 450, 530), (777, '2016-04-20', 420, 530, 640),
   (1234, '2016-11-15', 730, 740, 710), (1234, '2016-10-10', 340, 350, 360),
   (7899, '2016-10-02', 550, 750, 780), (7899, '2016-09-02', 650, 770, 790),
   (9981, '2016-01-01', 500, 400, 300), (9981, '2015-01-01', 550, 350, 400),
   (4422, '2016-07-04', 720, 740, 750), (4422, '2016-05-04', 650, 750, NULL),
   (8675309, '2012-04-29', 500, 500, 510), (8675309, '2012-02-29', 600, NULL, 610),
   (666, '2006-06-06', 666, 666, 666), (5555, '2016-08-12', 600, 500, NULL);

In case you want to determine what the correct results should be (which isn't hard with this small of a table):

In [ ]:
%%sql
SELECT * FROM students;

In [ ]:
%%sql

SELECT * FROM scores ORDER BY studentid, datetaken;

Given the above tables for student demographics and (presumably SAT) scores, let's look at extracting some information about how people performed.

Question 1:

Find how many students are from the west coast (defined as state being CA, WA, OR)

Question 2:

Find the composite score (best in each category) for each student

Question 3:

Okay, done with the warm up stuff.  Now find the most recent math score for each student

Note that these next two assume that you were asked the previous three, otherwise they'd be cruel to ask in an interview (they build on these answers)

Question 4:

Find the average best verbal (so, best for each student, then averaged over students) score (two separate queries):
 - for each age (which presumably correlates with grade level)
 - for each state

Question 5:

List all states with (separate queries):
 - any math score above 700
 - an average best math score above 650

Solutions appear below.  No peeking!  
*  
*  
*  
*  
*  
*  
*  
*  
*  
*  
*  
*  
*  
*  
*  
*  
*  
*  
*  


### Question 1

In [ ]:
%%sql

SELECT count(*) FROM students WHERE state in ('CA','WA','OR');


### Question 2

In [ ]:
%%sql

SELECT studentid, MAX(math), MAX(verbal), MAX(writing),  
MAX(math) + MAX(verbal) + MAX(writing) AS composite
FROM scores GROUP BY 1;

### Question 3

In [ ]:
%%sql

SELECT studentid, math FROM (SELECT * FROM (SELECT studentid, MAX(datetaken) AS recent FROM scores 
                                   WHERE math IS NOT NULL GROUP BY 1) AS a 
JOIN scores ON a.studentid = scores.studentid AND a.recent = scores.datetaken)

### Question 4

In [ ]:
%%sql

WITH best AS (SELECT studentid, MAX(math) AS bestmath, MAX(verbal) AS bestverbal, 
              MAX(writing) AS bestwriting FROM scores GROUP BY 1)
SELECT state, AVG(bestmath) FROM students join best ON students.studentid = best.studentid
GROUP BY 1

In [ ]:
%%sql

WITH best AS (SELECT studentid, MAX(math) AS bestmath, MAX(verbal) AS bestverbal, 
              MAX(writing) AS bestwriting FROM scores GROUP BY 1)
SELECT age, AVG(bestmath) FROM students join best ON students.studentid = best.studentid
GROUP BY 1

### Question 5

In [ ]:
%%sql

SELECT distinct state FROM students join scores ON students.studentid = scores.studentid
WHERE math > 700

In [ ]:
%%sql

WITH best AS (SELECT studentid, MAX(math) AS bestmath, MAX(verbal) AS bestverbal, 
              MAX(writing) AS bestwriting FROM scores GROUP BY 1)
SELECT state, AVG(bestmath) FROM students join best ON students.studentid = best.studentid
GROUP BY 1
HAVING AVG(bestmath) > 650

*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*